In the final project you will develop your own classification algorithm based on probabilistic models.

In [43]:
with open("DataFP/Spanish.txt") as f:
    print(f.read(500))

tomas zapata sierra medellin colombia  de mayo de  es productor de cine y de teatrocita requerida
sad eyed lady of the lowlands en espanol senorita de ojos tristes de las tierras bajas es una cancion compuesta por el cantante estadounidense bob dylan fue incluida en el album blonde on blonde editado el  de mayo de 
la revista mojo la coloco en el puesto  de su lista de las  mejores canciones de bob dylan
calyptocephalella canqueli es una especie extinta de anfibio anuro perteneciente al genero c


In [44]:
with open("C:/Users/User/Downloads/DataFP/German.txt") as f:
    print(f.read(500))

riedhofe ist der name von ortsteilen in deutschland

in badenwurttemberg
riedhofe langenau ortsteil der stadt langenau im albdonaukreis
riedhofe frickingen ortsteil der gemeinde frickingen im bodenseekreis
riedhofe riegel am kaiserstuhl ortsteil der gemeinde riegel am kaiserstuhl im landkreis emmendingen
riedhofe kongen ortsteil der gemeinde kongen im landkreis esslingen
riedhofe leingarten ortsteil der gemeinde leingarten im landkreis heilbronn
riedhofe bad wurzach ortsteil der stadt bad wurzac


This is our training data. These texts are preprocessed: only standard Latin characters kept, diacritics removed, punctuations removed, all letters converted to lowercase. We will use similar preprocessing for new texts that we have to classify.

Preprocessing functions

In [45]:
import re
import unicodedata

### FROM: https://stackoverflow.com/a/518232/3025981
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
### END FROM

def clean_text(s):
    return re.sub("[^a-z \n]", "", strip_accents(s))


First of all, we have to find character relative frequencies in texts of each language. We will consider them as probability for character to appear in our multinomial model.

In [46]:
from collections import Counter
def get_freqs(text, relative=False):
    return dict(Counter(list(text)))

In [49]:
lang_to_probs = {}
lan = ('Polish', 'English', 'Italian', 'Spanish', 'German', 'French', 'Portuguese')

for lang in lan:
    with open('C:/Users/User/Downloads/DataFP/' + lang + '.txt') as f:
        language = clean_text(strip_accents(f.read()))
        freqs = get_freqs(language)
        total_chars = sum(freqs.values())
        lang_to_probs[lang] = {char: freq / total_chars for char, freq in freqs.items()}


Now let us start implementing the actual classifier.

Function return probability to obtain these absolute frequencies from multinomial distribution with given probabilities $P((X_1 = f_1) \cap (X_2 = f_2) \cap \ldots \cap (X_k = f_k))$ provided that $(X_1, \ldots, X_k)$ is a system of multinomially distributed values with probabilities $(p_1, \ldots, p_k)$. 


In [51]:
import math

def multinomial_likelihood(probs, freqs):
    coef = math.factorial(sum(freqs.values()))
    for freq in freqs.values():
        coef /= math.factorial(freq)

    prob = coef
    for symb, freq in freqs.items():
        prob *= (probs[symb] ** freq)
    
    return prob

Let's find likelihood of data with frequencies `{'a': 2, 'b': 1, 'c': 2}` and probabilities `{'a': 0.2, 'b': 0.5, 'c': 0.3}`:

In [52]:
multinomial_likelihood(probs={'a': 0.2, 'b': 0.5, 'c': 0.3}, freqs={'a': 2, 'b': 1, 'c': 2})

0.054000000000000006

Note that the coefficient with factorials depends only on `freqs` and does not depend on `probs`. It means that for all possible languages this coefficient will be the same.

In [54]:
def multinomial_likelihood_without_coeff(probs, freqs):
    prob = 1
    for symb, freq in freqs.items():
        prob *= (probs[symb] ** freq)
    return prob    

Corresponding probability become smaller:

In [55]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                                     freqs={'a': 2, 'b': 1, 'c': 2})

0.0018000000000000004

Actually, likelihoods become extremely small very quickly when we increase absolute frequencies in data. It is not surprisingly: the probability to get the text that coincides with our actual text from random experiment we discussed is extremely small.

In [57]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 3, 'b': 2, 'c': 2})

0.00018000000000000004

In [58]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 3, 'b': 20, 'c': 2})

6.866455078125001e-10

Due to limited precision of computer arithmetic, for frequencies large enough we will get exactly zero likelihood.

In [59]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                                     freqs={'a': 543, 'b': 512, 'c': 2})

0.0

Thus we usually cannot use likelihoods like this directly. Common way to deal with such a tiny numbers is to use _logarithms_ instead of likelihood themself. Indeed, logarithm is monotonically increasing function. If we compare logarithms, it is equivalent to compare their arguments.


In [60]:
def log_likelihood_without_coeff(probs, freqs):
    prob = 0
    for symb, freq in freqs.items():
        prob += freq * math.log(probs[symb])
    return prob 

Likelihoods are probabilities, so they are less than 1 and their logarithms are negative. The larger absolute value of log-likelihood, the less is likelihood. 

In [61]:
log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 2, 'b': 1, 'c': 2})

-6.319968614080018

Now we can deal with inputs that lead to exact zero value previously.

In [62]:
log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 543, 'b': 512, 'c': 2})

-1231.2240885070605

Now we will use likelihood to choose the best language for some text we want to classify. 

In [64]:
def mle_best(text, lang_to_probs):
    text = clean_text(strip_accents(text))
    freqs = get_freqs(text)
    res = {}
    for k, v in lang_to_probs.items():
        res[k] = res.get(k, log_likelihood_without_coeff(v, freqs)) 
        
    return max(res.items(), key=lambda x: x[1])[0]

In [65]:
# Source: https://en.wikipedia.org/wiki/1134_Kepler
text = """1134 Kepler, provisional designation 1929 SA, is a stony asteroid 
and eccentric Mars-crosser from the asteroid belt, approximately 
4 kilometers in diameter"""
mle_best(text, lang_to_probs) #'English'


'English'

In [66]:
# Source: https://pl.wikipedia.org/wiki/(1134)_Kepler
text = """"(1134) Kepler – planetoida z grupy przecinających 
orbitę Marsa okrążająca Słońce w ciągu 4 lat i 145 dni 
w średniej odległości 2,68 au.
"""
mle_best(text, lang_to_probs)

'Polish'

In [67]:
# Source: https://it.wikipedia.org/wiki/1134_Kepler
text = """1134 Kepler è un asteroide areosecante. Scoperto nel 1929, 
presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 
UA e da un'eccentricità di 0,4651458, inclinata di 15,17381° rispetto
"""
mle_best(text, lang_to_probs)

'Italian'

Assume that we selected random article from all Wikipedia articles in languages that we consider. There are different number of articles in different languages, so it is more likely to obtain article e.g. in English than in Polish (at least, at this moment). It means that we need stronger evidence for Polish to accept it comparing with English. To take into account this information, we will use Bayes' rule as discussed in the videos.

Recall that in Bayessian approach we consider _prior_ probabilities of languages, then use Bayes' rule to find their posterior probabilies and select language with largest posterior probability.

In [69]:
lang_to_prior = {'English': 6090/16410, 'Italian': 1611/16410, 'Spanish': 1602/16410, 'German': 2439/16410, 
                 'French': 2222/16410, 'Polish': 1412/16410, 'Portuguese': 1034/16410}

In [71]:
def bayesian_best(text, lang_to_probs, lang_to_prior):
    text = clean_text(strip_accents(text))
    freqs = get_freqs(text)
    res = {}
    for k, v in lang_to_probs.items():
        res[k] = res.get(k, log_likelihood_without_coeff(v, freqs)+ math.log(lang_to_prior[k])) 
        
    return max(res.items(), key=lambda x: x[1])[0]    


For example, MLE algorithm believes that word `"The"` belongs go German language.

In [72]:
mle_best("The", lang_to_probs)

'German'

However, if we take into account that English is more popular in Wikipdia, results changes:

In [73]:
bayesian_best("The", lang_to_probs, lang_to_prior)

Finally, let us get posterior probability how certain our Bayesian algorithm in its classification

In [75]:
def bayesian_posterior(text, lang_to_probs, lang_to_prior, test_lang):
    text = clean_text(strip_accents(text))
    freqs = get_freqs(text)
    posterior = {}
    
    for lang, probs in lang_to_probs.items():
        likelihood = multinomial_likelihood(probs, freqs)
        posterior[lang] = likelihood * lang_to_prior[lang]
        
    total_posterior = sum(posterior.values())
    for lang in posterior:
        posterior[lang] /= total_posterior
    
    return posterior[test_lang]


In [76]:
bayesian_posterior("The", lang_to_probs, lang_to_prior, "German")

In [77]:
bayesian_posterior("The", lang_to_probs, lang_to_prior, "English")

In [78]:
bayesian_posterior("Das", lang_to_probs, lang_to_prior, "English")

In [79]:
bayesian_posterior("Das", lang_to_probs, lang_to_prior, "German")

We see that our algorithm believes that `"Das"` belongs to English. This is due to small amount of data (only three letters!) and high prior for English. However, it is not very certain: the posterior is only 0.37. On the other hand, `"The"` belongs to `"English"` with much larger posterior probability.